In [14]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [18]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\output data'
name_DD_M = r'dataASCD_M.xlsx'
name_DD_P = r'dataASCD_P.xlsx'
ruteDD_M = os.path.join(folder,name_DD_M)
ruteDD_P = os.path.join(folder,name_DD_P)


In [23]:
dataASCD_M = pd.read_excel(ruteDD_M, header=0)
dataASCD_P = pd.read_excel(ruteDD_P, header=0)

In [25]:
dataASCD_M.shape

(3276, 28)